In [1]:
import collections
import logging
from copy import deepcopy

from intcode import Machine

In [2]:
with open("day15.input") as file:
    prog = file.readline().strip()

In [3]:
def draw_grid(grid):
    max_x = max((k[0] for k in grid))
    min_x = min((k[0] for k in grid))
    max_y = max((k[1] for k in grid))
    min_y = min((k[1] for k in grid))

    for x in range(max_x, min_x - 1, -1):
        for y in range(min_y, max_y + 1):
            if grid.get((x, y)):
                print(grid.get((x, y)), end='')
            else:
                print(' ', end='')
        print()

In [4]:
def update_pos(pos, move):
    #  1
    # 3 4
    #  2
    if move == 1:
        return (pos[0] + 1, pos[1])
    if move == 2:
        return (pos[0] - 1, pos[1])
    if move == 3:
        return (pos[0], pos[1] - 1)
    if move == 4:
        return (pos[0], pos[1] + 1)
    raise ValueError("Illegal move:".format(move))

In [5]:
def find_neighbours(pos, machine):

    # How to reverse a move
    back = {
        1: 2,
        2: 1,
        3: 4,
        4: 3
    }

    neighbours = []
    for move in range(1, 5):

        # Have we already visited the candidate?
        n_pos = update_pos(pos, move)
        if grid.get(n_pos):
            continue

        # Try to move in a direction
        machine.add_input(move)
        machine.run()
        output = machine.get_output()
            
        # Check resulting output
        if output == 0:
            grid[n_pos] = '▒'
        elif output == 1:
            # It was possible to move in this direction
            grid[n_pos] = '·'
            neighbours.append((n_pos, deepcopy(machine), False))
        elif output == 2:
            # It was possible to move in this direction
            # And this is the oxygen system
            grid[n_pos] = '★'
            neighbours.insert(0, (n_pos, deepcopy(machine), True))
            
        # If we moved: Move back before trying the next move
        if output in [1, 2]:
            machine.add_input(back[move])
            machine.run()
            _ = machine.get_output()

    return neighbours

In [6]:
def bfs(pos, robot):
    target_pos = None
    m_at_target = None
    visited = set()
    dist = dict()
    dist[start] = 0
    backtrack = dict()
    backtrack[start] = None
    queue = collections.deque([(pos, robot)])

    while queue:
        pos, robot = queue.pop()
        visited.add(pos)
        
        for n, m, target in find_neighbours(pos, robot):
            dist[n] = dist[pos] + 1
            backtrack[n] = pos
            if target:
                print("Target found: {} at distance {}".format(n, dist[n]))
                target_pos = n
                m_at_target = m
            if n not in visited:
                queue.appendleft((n, m))

    return target_pos, m_at_target, dist, backtrack

# Part 1

In [13]:
r = Machine(prog, loglevel=logging.WARNING)

start = (0, 0)
grid = dict()
grid[start] = 'S'

target_pos, m_at_target, dist, backtrack = bfs(start, r)

Target found: (16, 16) at distance 236


In [14]:
dist[target_pos]

236

In [15]:
p = target_pos
while backtrack[p]:
    grid[p] = 'o'
    p = backtrack[p]
grid[target_pos] = '☆'

In [16]:
draw_grid(grid)

 ▒▒▒▒▒▒▒ ▒▒▒▒▒▒▒▒▒ ▒▒▒▒▒▒▒▒▒ ▒▒▒ ▒▒▒▒▒ ▒ 
▒·······▒·········▒ooooooooo▒···▒··ooo▒·▒
▒·▒▒▒·▒▒▒·▒·▒▒▒▒▒·▒o▒▒▒▒▒▒▒o▒▒▒·▒·▒o▒o▒·▒
▒···▒·▒···▒···▒···▒ooo▒···▒ooo▒···▒o▒ooo▒
 ▒▒·▒·▒·▒▒ ▒▒·▒▒▒·▒▒▒o▒▒▒·▒▒▒o▒▒▒▒▒o▒▒▒o▒
▒···▒·▒···▒·▒···▒···▒ooo▒ooooo▒ooooo▒☆oo▒
▒·▒▒▒▒ ▒▒·▒·▒▒▒·▒·▒▒▒▒▒o▒o▒▒▒▒▒o▒▒▒▒▒▒▒·▒
▒·▒···▒·······▒·▒·▒ooooo▒ooooo▒o▒ooooo▒·▒
▒·▒·▒·▒·▒▒▒▒▒▒▒·▒·▒o▒▒▒▒▒▒▒▒▒o▒o▒o▒▒▒o▒·▒
▒·▒·▒·····▒·····▒·▒o▒·▒ooooooo▒ooo▒·▒o▒·▒
▒·▒·▒▒▒▒▒▒▒·▒▒▒▒▒·▒o▒·▒o▒▒▒▒▒▒▒·▒▒▒·▒o▒·▒
▒···▒·····▒·▒·····▒o▒ooo▒·····▒·····▒o▒·▒
▒·▒▒▒·▒▒▒·▒·▒·▒▒▒▒▒o▒o▒▒▒·▒·▒▒▒▒▒·▒▒▒o▒·▒
▒···▒·▒·▒·▒·▒···▒ooo▒ooo▒·▒·▒·····▒ooo▒·▒
 ▒▒·▒·▒·▒·▒·▒▒▒·▒o▒▒▒▒▒o▒·▒·▒·▒▒▒▒▒o▒▒▒·▒
▒···▒·▒·▒·▒·▒···▒o▒···▒o▒·▒·······▒o▒···▒
▒·▒▒▒·▒·▒·▒·▒·▒▒▒o▒·▒▒▒o▒·▒▒▒▒▒▒▒▒▒o▒·▒▒ 
▒·▒·▒·▒···▒·▒·▒·▒o▒···▒o▒···▒ooooooo▒·▒·▒
▒·▒·▒·▒·▒▒▒·▒·▒·▒o▒▒▒·▒o▒▒▒▒▒o▒▒▒▒▒▒▒·▒·▒
▒·▒·▒·▒···▒·▒···▒ooo▒··ooo▒ooo▒···▒···▒·▒
▒·▒·▒·▒▒▒·▒·▒▒▒▒▒▒▒o▒▒▒▒▒o▒o▒▒▒·▒·▒·▒▒▒·▒
▒···▒·▒·▒·▒·······▒o▒S▒ooo▒ooo▒·▒···▒···▒
 ▒▒·▒·▒·▒·▒▒▒▒▒▒▒·▒o▒o▒o▒▒▒·▒o▒▒▒▒▒·▒·▒·▒
▒···▒·▒·········▒·▒o▒o▒o▒···▒ooooo

# Part 2

In [26]:
# Start at the oxygen outlet and do a BFS
start = target_pos
grid = dict()
grid[start] = 'O'
_, _, dist, backtrack = bfs(start, m_at_target)

In [27]:
max(dist.values())

368